## <a id='toc1_1_'></a>[Data Loading and Cleaning](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Data Loading and Cleaning](#toc1_1_)    
    - [Introduction](#toc1_1_1_)    
      - [Downloading Amazon dataset](#toc1_1_1_1_)    
      - [Importing Python Libraries](#toc1_1_1_2_)    
    - [Loading the meta dataset](#toc1_1_2_)    
      - [Categories column](#toc1_1_2_1_)    
      - [Details Column](#toc1_1_2_2_)    
      - [Department Column](#toc1_1_2_3_)    
    - [Loading Review dataset](#toc1_1_3_)    
    - [Conclusion](#toc1_1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_1_'></a>[Introduction](#toc0_)

In this notebook, we load the Amazon dataset for Handmade Items and perform additional cleaning on the product metadata and reviews dataset. Our goal is to extract relevant features suitable for further preprocessing and exploratory analysis.

#### <a id='toc1_1_1_1_'></a>[Downloading Amazon dataset](#toc0_)

You can download the necessary dataset from [here](https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw). Each product category includes both metadata and review data. For this project, I focus on the Handmade Products category.

#### <a id='toc1_1_1_2_'></a>[Importing Python Libraries](#toc0_)

Importing necessary libraries for data pre-processing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re

# # Ignore all warnings to avoid cluttering the output
import warnings
warnings.filterwarnings("ignore")

plt.rcParams['figure.figsize'] = (8.0, 6.0) # set matplotlib global settings eg. figsize
sns.set_style("darkgrid")  #Setting grid style in seaborn

### <a id='toc1_1_2_'></a>[Loading the meta dataset](#toc0_)

Overview: Contains item meta data including average_rating, rating_number, features etc.

Step: Importing meta data into a new dataframe meta_df.

In [2]:
# Load the Meta data json file into a dataframe
meta_df = pd.read_json('../data/meta_Handmade_Products.jsonl',lines=True)

# Display the first few rows of the 'meta_df' DataFrame
meta_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,Handmade,Daisy Keychain Wristlet Gray Fabric Key fob La...,4.5,12,"[High Quality Fabrics, Antique Brass Metallic ...",[This charming Daisy Fabric Keychain wristlet ...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,"[Handmade Products, Clothing, Shoes & Accessor...",{'Package Dimensions': '8 x 4 x 0.85 inches; 0...,B07NTK7T5P,NaN
1,Handmade,Anemone Jewelry Beauteous November Birthstone ...,4.1,10,"[Stunning gemstone and detailed design, Bands ...",[Anemone brings this November birthstone ring ...,69.00,[{'thumb': 'https://m.media-amazon.com/images/...,[],Anemone Jewelry,"[Handmade Products, Jewelry, Rings, Statement]","{'Department': 'womens', 'Date First Available...",B0751M85FV,NaN
2,Handmade,Silver Triangle Earrings with Chevron Pattern,5.0,1,[],[These large silver triangles are stamped with...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Zoë Noelle Designs,"[Handmade Products, Jewelry, Earrings, Drop & ...","{'Department': 'Women', 'Date First Available'...",B01HYNE114,NaN
3,Handmade,"Las Vegas Subway Sign Print - Caesar's, Freemo...",1.0,1,[],[Subway Signs make fabulous wall art. A bit of...,19.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,"[Handmade Products, Home & Kitchen, Artwork, P...","{'Date First Available': 'June 14, 2018'}",B07TKZF3Z1,NaN
4,Handmade,Round Cut Cubic Zirconia Stud Earrings Fashion...,4.2,2,[⭐【Size】The remarkable cubic zirconia studs ha...,[],14.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],VDKIDKT,"[Handmade Products, Jewelry, Earrings, Stud]",{'Package Dimensions': '2.36 x 2.05 x 1.65 inc...,B0BKBJT5MM,NaN


In [3]:
# Inspecting meta_df and check its concise summary
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164817 entries, 0 to 164816
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   main_category    164803 non-null  object 
 1   title            164817 non-null  object 
 2   average_rating   164817 non-null  float64
 3   rating_number    164817 non-null  int64  
 4   features         164817 non-null  object 
 5   description      164817 non-null  object 
 6   price            97648 non-null   float64
 7   images           164817 non-null  object 
 8   videos           164817 non-null  object 
 9   store            163699 non-null  object 
 10  categories       164817 non-null  object 
 11  details          164817 non-null  object 
 12  parent_asin      164817 non-null  object 
 13  bought_together  0 non-null       float64
dtypes: float64(3), int64(1), object(10)
memory usage: 17.6+ MB


In [4]:
# Print the shape of the 'meta_df' DataFrame
print(f"The shape of the amazon meta dataset is {meta_df.shape[0]} by {meta_df.shape[1]}.")

The shape of the amazon meta dataset is 164817 by 14.


In [5]:
# Check duplicated rows (after removing columns that contain lists)
duplicated_rows = meta_df.drop(columns=['categories','features','description','details','images','videos']).duplicated().sum()

print(f'There are {duplicated_rows} duplicated rows in the dataset.')

There are 0 duplicated rows in the dataset.


In [6]:
#check the proportion of null values 
meta_df.isnull().mean()*100

main_category        0.008494
title                0.000000
average_rating       0.000000
rating_number        0.000000
features             0.000000
description          0.000000
price               40.753684
images               0.000000
videos               0.000000
store                0.678328
categories           0.000000
details              0.000000
parent_asin          0.000000
bought_together    100.000000
dtype: float64

Since the `bought_together` column contains only null values, we drop it from the dataframe.

In [7]:
meta_df.drop(columns='bought_together',inplace=True)

We are missing less than 1% of the data in the `main_category` and `store` columns in the meta dataset. Since the missing values are minimal, we will proceed by dropping them.

In [8]:
meta_df = meta_df.dropna(subset=['main_category','store']).reset_index(drop=True)

In [9]:
#Checking if all items belong to the category of `Hand made Products`
meta_df['main_category'].value_counts(normalize=True,dropna=False) 

main_category
Handmade                  0.999768
Amazon Home               0.000067
AMAZON FASHION            0.000055
Office Products           0.000037
Health & Personal Care    0.000018
Pet Supplies              0.000018
All Beauty                0.000012
All Electronics           0.000012
Arts, Crafts & Sewing     0.000006
Amazon Devices            0.000006
Name: proportion, dtype: float64

Since the vast majority of the data (99.98%) belongs to the Handmade category, and the remaining categories make up only a tiny fraction of the dataset, we will remove these other categories to ensure we are working with a single, consistent category.

In [10]:
meta_handmade_df = meta_df[meta_df['main_category'] == 'Handmade'].reset_index(drop=True)
meta_handmade_df['main_category'].value_counts(dropna= False)

main_category
Handmade    163647
Name: count, dtype: int64

In [11]:
meta_handmade_df['categories'].head()

0    [Handmade Products, Clothing, Shoes & Accessor...
1       [Handmade Products, Jewelry, Rings, Statement]
2    [Handmade Products, Jewelry, Earrings, Drop & ...
3    [Handmade Products, Home & Kitchen, Artwork, P...
4         [Handmade Products, Jewelry, Earrings, Stud]
Name: categories, dtype: object

The `categories` column contains list of hierarchical product categories. Lets see if there are any products with missing categories by examing the length of the lists in the column.

In [12]:
np.sort(meta_handmade_df['categories'].map(len).unique())

array([0, 2, 3, 4, 5, 6, 7])

In [13]:
print(f"There are {len(meta_handmade_df[meta_handmade_df['categories'].map(len) == 0])} records with missing categories.")


There are 1030 records with missing categories.


In [14]:
# Delete rows corresponding to missing categories to ensure we have category information for all the products.
meta_handmade_df = meta_handmade_df[meta_handmade_df['categories'].map(len) > 0].reset_index(drop=True)

#sanity check
print(f"There are {len(meta_handmade_df[meta_handmade_df['categories'].map(len) == 0])} records with missing categories.")

There are 0 records with missing categories.


`Price` column  has over 40% of the rows with missing data. For the purposes of this project, we will be using a sample of our dataframe that do not contain missing prices. 

In [15]:
#we work with a 50% of the dataset
meta_sample_df = meta_handmade_df[~meta_handmade_df['price'].isnull()].sample(frac=0.5,replace=False, random_state=1).reset_index(drop = True)

meta_sample_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin
0,Handmade,Sterling Silver Hammered Ear Cuff,4.4,243,"[Solid Sterling Silver, Artisan Handcrafted, C...",[This simple ear cuff is made with solid Sterl...,24.00,[{'thumb': 'https://m.media-amazon.com/images/...,[],Twisted Designs Jewelry,"[Handmade Products, Jewelry, Earrings, Ear Cuff]",{'Package Dimensions': '3.62 x 2.05 x 0.83 inc...,B0178HXZUY
1,Handmade,Witch Cigarette Case Business Card ID Holder W...,5.0,1,[],[This silver-tone metal case can be used as a ...,12.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,"[Handmade Products, Clothing, Shoes & Accessor...","{'Department': 'unisex-adult', 'Date First Ava...",B07B8ZKCVM
2,Handmade,"Humorous Cat Wall Art - Decor for Home, Office...",4.5,108,[8x10 UNFRAMED PRINTS - NOT Canvas or Tin Sign...,[8x10 UNFRAMED Prints],12.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],YELLOWBIRD ART & DESIGN,"[Handmade Products, Home & Kitchen, Artwork, P...",{'Package Dimensions': '11.2 x 8.8 x 0.2 inche...,B07ZFJXDH8
3,Handmade,Whiskey Glasses by Black Lantern – Floral Whis...,4.4,11,[PREMIUM GLASSWARE READY TO GIFT - Our glasses...,[],31.00,[{'thumb': 'https://m.media-amazon.com/images/...,[],Black Lantern,"[Handmade Products, Home & Kitchen, Kitchen & ...","{'Package Dimensions': '9 x 7 x 5 inches', 'Da...",B089LRPX7X
4,Handmade,LOVE Dog Paw Print Heart Sticker Decal Compati...,4.1,3,[],"[Decal Size: 1"" Tall X 3.2"" Wide Die cut out o...",3.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,"[Handmade Products, Electronics Accessories, L...","{'Department': 'Womens', 'Date First Available...",B01MXKS1L5


In [16]:
# Print the shape of the 'meta_sample_df' DataFrame
print(f"The shape of the sampled meta dataset is {meta_sample_df.shape[0]} by {meta_sample_df.shape[1]}.")

The shape of the sampled meta dataset is 48576 by 13.


In [17]:
#checking info
meta_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48576 entries, 0 to 48575
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   main_category   48576 non-null  object 
 1   title           48576 non-null  object 
 2   average_rating  48576 non-null  float64
 3   rating_number   48576 non-null  int64  
 4   features        48576 non-null  object 
 5   description     48576 non-null  object 
 6   price           48576 non-null  float64
 7   images          48576 non-null  object 
 8   videos          48576 non-null  object 
 9   store           48576 non-null  object 
 10  categories      48576 non-null  object 
 11  details         48576 non-null  object 
 12  parent_asin     48576 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 4.8+ MB


While there are no longer any null values in the dataset, columns `images`, `videos`, `features`, `description` contain lists, with [] representing null values. We will deal with the missing values in these columns during pre-processing.

Since `categories` column represents the hierarchical categories of the product, we can further split it into subcategory levels.

#### <a id='toc1_1_2_1_'></a>[Categories column](#toc0_)

In [18]:
#split the list of categories column into different columns in the dataframe
col_names_dict = {0:'maincategory',
                1:'sub_category_1',
                2:'sub_category_2',
                3:'sub_category_3',
                4:'sub_category_4',
                5:'sub_category_5',
                6:'sub_category_6'}

meta_sample_df = meta_sample_df.join(meta_sample_df.apply(lambda x: pd.Series(x['categories']), axis=1)).rename(columns=col_names_dict)

#The first sub-category is simply the main category, so dropping it from the dataset
meta_sample_df.drop(columns='maincategory',inplace=True)

meta_sample_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,sub_category_1,sub_category_2,sub_category_3,sub_category_4,sub_category_5,sub_category_6
0,Handmade,Sterling Silver Hammered Ear Cuff,4.4,243,"[Solid Sterling Silver, Artisan Handcrafted, C...",[This simple ear cuff is made with solid Sterl...,24.00,[{'thumb': 'https://m.media-amazon.com/images/...,[],Twisted Designs Jewelry,"[Handmade Products, Jewelry, Earrings, Ear Cuff]",{'Package Dimensions': '3.62 x 2.05 x 0.83 inc...,B0178HXZUY,Jewelry,Earrings,Ear Cuff,NaN,NaN,NaN
1,Handmade,Witch Cigarette Case Business Card ID Holder W...,5.0,1,[],[This silver-tone metal case can be used as a ...,12.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,"[Handmade Products, Clothing, Shoes & Accessor...","{'Department': 'unisex-adult', 'Date First Ava...",B07B8ZKCVM,"Clothing, Shoes & Accessories",Luggage & Travel Gear,Wallets & Bag Accessories,Cigarette Cases,NaN,NaN
2,Handmade,"Humorous Cat Wall Art - Decor for Home, Office...",4.5,108,[8x10 UNFRAMED PRINTS - NOT Canvas or Tin Sign...,[8x10 UNFRAMED Prints],12.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],YELLOWBIRD ART & DESIGN,"[Handmade Products, Home & Kitchen, Artwork, P...",{'Package Dimensions': '11.2 x 8.8 x 0.2 inche...,B07ZFJXDH8,Home & Kitchen,Artwork,Prints,NaN,NaN,NaN
3,Handmade,Whiskey Glasses by Black Lantern – Floral Whis...,4.4,11,[PREMIUM GLASSWARE READY TO GIFT - Our glasses...,[],31.00,[{'thumb': 'https://m.media-amazon.com/images/...,[],Black Lantern,"[Handmade Products, Home & Kitchen, Kitchen & ...","{'Package Dimensions': '9 x 7 x 5 inches', 'Da...",B089LRPX7X,Home & Kitchen,Kitchen & Dining,Tableware,Glassware,Tumblers,NaN
4,Handmade,LOVE Dog Paw Print Heart Sticker Decal Compati...,4.1,3,[],"[Decal Size: 1"" Tall X 3.2"" Wide Die cut out o...",3.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,"[Handmade Products, Electronics Accessories, L...","{'Department': 'Womens', 'Date First Available...",B01MXKS1L5,Electronics Accessories,Laptop Accessories,Skins & Decals,NaN,NaN,NaN


In [19]:
#Checking null values
meta_sample_df.isna().mean()*100

main_category      0.000000
title              0.000000
average_rating     0.000000
rating_number      0.000000
features           0.000000
description        0.000000
price              0.000000
images             0.000000
videos             0.000000
store              0.000000
categories         0.000000
details            0.000000
parent_asin        0.000000
sub_category_1     0.000000
sub_category_2     0.150280
sub_category_3     6.414690
sub_category_4    72.770504
sub_category_5    94.069088
sub_category_6    99.921772
dtype: float64

Subcategory levels 4, 5, and 6 have the highest percentage of missing values, making them less reliable for analysis. For products with valid labels at these levels, we attempt to integrate them into earlier levels where possible.

In [20]:
#deleting rows with null values in sub_category_2 as the missing data is < 1%
meta_sample_df.dropna(subset='sub_category_2', inplace=True)

In [21]:
#Checking labels under sub_category1
meta_sample_df['sub_category_1'].value_counts()

sub_category_1
Home & Kitchen                                 20009
Jewelry                                        15022
Clothing, Shoes & Accessories                   4799
Stationery & Party Supplies                     3933
Beauty & Grooming                               1274
Sports & Outdoors                                770
Last minute gifts                                662
Electronics Accessories                          571
Pet Supplies                                     451
Toys & Games                                     375
Baby                                             331
Health & Personal Care                           266
Handmade Small Business Promotion - Jewelry       26
Handmade Small Business Promotion                  6
Southeast States                                   4
Northeast States                                   2
Southwest States                                   1
Midwest States                                     1
Name: count, dtype: int64

Regional labels (e.g., Southeast States, Northeast States) and promotional event-based labels (e.g., Handmade Small Business Promotion) do not categorize products based on their characteristics but rather indicate geographic or marketing-related aspects. To maintain consistency in category definitions, we will remove rows containing these labels.

In [22]:
#Remove rows with regional labels
meta_sample_df = meta_sample_df[~meta_sample_df['sub_category_1'].str.contains('States|Handmade|Prime')]

meta_sample_df['sub_category_1'].value_counts().index.unique().to_list()

['Home & Kitchen',
 'Jewelry',
 'Clothing, Shoes & Accessories',
 'Stationery & Party Supplies',
 'Beauty & Grooming',
 'Sports & Outdoors',
 'Last minute gifts',
 'Electronics Accessories',
 'Pet Supplies',
 'Toys & Games',
 'Baby',
 'Health & Personal Care']

In [23]:
#Checking labels under 'Last minute gifts'
last_minute_gifts = meta_sample_df[meta_sample_df['sub_category_1']=='Last minute gifts']
last_minute_gifts['sub_category_2'].value_counts(dropna=False)

sub_category_2
Toys & Games              245
Beauty & Grooming         128
Home & Kitchen             91
Pet Supplies               65
Sporting Goods             53
Baby                       40
Handbags & Accessories     37
Clothing                    2
Shoes                       1
Name: count, dtype: int64

Since ‘Last Minute Gifts’ is a broad and non-specific label, we choose to rely on the more meaningful sub_category_2 labels for these products. The sub_category_2 labels (e.g., ‘Toys & Games,’ ‘Beauty & Grooming,’ etc.) provide clearer distinctions between product types, making them more useful for modeling and analysis.

In [24]:
# Select rows where sub_category_1 is 'Last minute gifts'
mask = meta_sample_df['sub_category_1'] == 'Last minute gifts'

# Shift subcategories by 1 level for selected rows
meta_sample_df.loc[mask, 'sub_category_1'] = meta_sample_df.loc[mask, 'sub_category_2']
meta_sample_df.loc[mask, 'sub_category_2'] = meta_sample_df.loc[mask, 'sub_category_3']
meta_sample_df.loc[mask, 'sub_category_3'] = meta_sample_df.loc[mask, 'sub_category_4']
meta_sample_df.loc[mask, 'sub_category_4'] = meta_sample_df.loc[mask, 'sub_category_5']
meta_sample_df.loc[mask, 'sub_category_5'] = meta_sample_df.loc[mask, 'sub_category_6']

# Optionally, set the last category to NaN since it shifts out of scope
meta_sample_df.loc[mask, 'sub_category_6'] = np.nan

meta_sample_df['sub_category_1'].value_counts()

sub_category_1
Home & Kitchen                   20100
Jewelry                          15022
Clothing, Shoes & Accessories     4799
Stationery & Party Supplies       3933
Beauty & Grooming                 1402
Sports & Outdoors                  770
Toys & Games                       620
Electronics Accessories            571
Pet Supplies                       516
Baby                               371
Health & Personal Care             266
Sporting Goods                      53
Handbags & Accessories              37
Clothing                             2
Shoes                                1
Name: count, dtype: int64

In [25]:
# Lets check null values introduced in sub_category_2 because of shfting categories above
print(f'There are {meta_sample_df['sub_category_2'].isna().sum()} null values in the sub_category_2 column of the dataframe.')

There are 662 null values in the sub_category_2 column of the dataframe.


In [26]:
#dropping nulls in sub_category2 since its a small number
meta_sample_df.dropna(subset='sub_category_2', inplace=True)

print(f'There are {meta_sample_df['sub_category_2'].isna().sum()} null values in the sub_category_2 column of the dataframe.')

There are 0 null values in the sub_category_2 column of the dataframe.


The "Handbags & Accessories" category is ambiguous, mixing unrelated products like electronics and clothing accessories. With only 37 rows, deleting them has minimal impact on the dataset while improving data quality and simplifying category structure. This ensures cleaner, more meaningful analysis and modeling.

In [27]:
#combining labels that are similar :
label_map = {'Sporting Goods': 'Sports & Outdoors',
             'Clothing':'Clothing, Shoes & Accessories',
             'Shoes': 'Clothing, Shoes & Accessories'}

meta_sample_df['sub_category_1'] = meta_sample_df['sub_category_1'].replace(label_map)

#removing ambiguous labels
meta_sample_df = meta_sample_df[~meta_sample_df['sub_category_1'].isin(["Handbags & Accessories","Birthday",
                                                                     "Wedding","Valentine's Day Gifts" ])].reset_index(drop=True)
meta_sample_df['sub_category_1'].value_counts()

sub_category_1
Home & Kitchen                   20009
Jewelry                          15022
Clothing, Shoes & Accessories     4799
Stationery & Party Supplies       3933
Beauty & Grooming                 1274
Sports & Outdoors                  770
Electronics Accessories            571
Pet Supplies                       451
Toys & Games                       375
Baby                               331
Health & Personal Care             266
Name: count, dtype: int64

In the following cells, we examine the subcategory2 labels within each subcategory1 group. We then refine the labels to ensure:
- They are descriptive enough to clearly convey the product category.
- Labels from subcategory levels 4, 5, and 6 are consolidated into the top three levels wherever possible to minimize information loss when those subcategories are removed.
- There are unique labels at each sub-category level.

In [28]:
home_and_kitchen_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Home & Kitchen']
home_and_kitchen_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Artwork                   8873
Home Décor                7504
Kitchen & Dining          2224
Storage & Organization     403
Patio, Lawn & Garden       384
Lighting                   277
Furniture                  171
Bath                        78
Cleaning Supplies           52
Bedding                     43
Name: count, dtype: int64

In [29]:
jewelry_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Jewelry']
jewelry_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Necklaces                        4825
Earrings                         3673
Bracelets                        2662
Rings                            2013
Body Jewelry                      631
Brooches, Buttons & Pins          404
Charms                            258
Jewelry Sets                      175
Prayer & Meditation Beads         138
Cufflinks & Shirt Accessories     131
Accessories                        80
Watches                            32
Name: count, dtype: int64

In [30]:
#Clothing, Shoes & Accessories
clothing_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Clothing, Shoes & Accessories']
clothing_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Luggage & Travel Gear         1734
Men                           1394
Women                          844
Handbags & Shoulder Bags       263
Boys                           235
Baby                           230
Girls                           55
Insoles & Shoe Accessories      44
Name: count, dtype: int64

In [31]:
#regrouping redundant sub-category labels to make them more descriptive

meta_sample_df.loc[meta_sample_df['sub_category_2']== 'Men','sub_category_2']= 'Men ' + \
                                                                            meta_sample_df['sub_category_3'].fillna('unknown')

meta_sample_df.loc[meta_sample_df['sub_category_2']== 'Women','sub_category_2']= 'Women ' + \
                                                                            meta_sample_df['sub_category_3'].fillna('unknown')

meta_sample_df.loc[meta_sample_df['sub_category_2']== 'Boys','sub_category_2']= 'Boys ' + \
                                                                            meta_sample_df['sub_category_3'].fillna('unknown')

meta_sample_df.loc[meta_sample_df['sub_category_2']== 'Girls','sub_category_2']= 'Girls ' + \
                                                                            meta_sample_df['sub_category_3'].fillna('unknown')

meta_sample_df.loc[meta_sample_df['sub_category_2']== 'Baby','sub_category_2']=  \
                                                                            meta_sample_df['sub_category_3'] + ' ' + \
                                                                            meta_sample_df['sub_category_4'].fillna('unknown')
                                                                            
clothing_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Clothing, Shoes & Accessories']
 

print(clothing_df['sub_category_2'].value_counts(dropna=False))


sub_category_2
Luggage & Travel Gear         1734
Men Clothing                  1021
Women Clothing                 429
Men Accessories                343
Women Accessories              311
Handbags & Shoulder Bags       263
Boys Clothing                  216
Baby Boys Clothing             145
Women Shoes                    104
Baby Girls Clothing             65
Girls Clothing                  49
Insoles & Shoe Accessories      44
Men Shoes                       30
Boys Accessories                19
Baby Boys unknown               11
Baby Boys Shoes                  7
Girls Accessories                4
Girls Shoes                      2
Baby Girls Shoes                 1
Baby Girls unknown               1
Name: count, dtype: int64


In [32]:
# Shifting sub_categories by 1 level for rows that had sub_category_2 labels renamed in the previous cell.
# Since sub_category_4,sub_category_5 and sub_category_6 will be deleted later, this ensures no meaningful data is lost.

target_categories = ['Men Clothing',
 'Women Clothing','Women Accessories',
 'Men Accessories', 'Boys Clothing',
 'Girls Clothing', 'Women Shoes',
 'Men Shoes','Boys Accessories']

# Replace sub_category_3 with sub_category_4 for the target categories 
meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(target_categories),
    'sub_category_3'
                  ] = meta_sample_df['sub_category_4']

#similarly replace the sub_category_4 labels with sub_category_5 ones
meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(target_categories),
    'sub_category_4'
                  ] = meta_sample_df['sub_category_5']

meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(target_categories),
    'sub_category_5'
                  ] = meta_sample_df['sub_category_6']
meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(target_categories),
    'sub_category_6'
                  ] = np.nan


baby_categories = ['Baby Boys Clothing', 'Baby Girls Clothing']

# Replace sub_category_3 with sub_category_5 for the baby categories 
meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(baby_categories),
    'sub_category_3'
                  ] = meta_sample_df['sub_category_5']

#similarly replace the sub_category_4 labels with sub_category_6 ones
meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(baby_categories),
    'sub_category_4'
                  ] = meta_sample_df['sub_category_6']

meta_sample_df.loc[
    meta_sample_df['sub_category_2'].isin(baby_categories),
    'sub_category_6'
                  ] = np.nan

In [33]:
#Deleting categories with very few items
clothing_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Clothing, Shoes & Accessories']

#Excluding sub_categories with fewer than 12 items
categories_to_exclude = clothing_df['sub_category_2'].value_counts().loc[lambda x: x < 12].index.to_list()

meta_sample_df = meta_sample_df[~meta_sample_df['sub_category_2'].isin(categories_to_exclude)].reset_index(drop=True)

#Checking if those categories have been deleted
clothing_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Clothing, Shoes & Accessories']
print(clothing_df['sub_category_2'].value_counts(dropna=False))


sub_category_2
Luggage & Travel Gear         1734
Men Clothing                  1021
Women Clothing                 429
Men Accessories                343
Women Accessories              311
Handbags & Shoulder Bags       263
Boys Clothing                  216
Baby Boys Clothing             145
Women Shoes                    104
Baby Girls Clothing             65
Girls Clothing                  49
Insoles & Shoe Accessories      44
Men Shoes                       30
Boys Accessories                19
Name: count, dtype: int64


Some sub-categories under different sub category level 2  have similar names - for eg. Tops & Tees could be found under both Men and Women Clothing, so lets update those.

In [34]:
# Identify sub_category_3 values that are associated with multiple sub_category_2 values
subcat2_counts = meta_sample_df.groupby('sub_category_3')['sub_category_2'].nunique()

# Find non-unique subcategories (appearing under multiple sub_category_2)
non_unique_subcat = subcat2_counts[subcat2_counts > 1].index

# Create masks for updating sub_category_3 labels
mask_non_unique = meta_sample_df['sub_category_3'].isin(non_unique_subcat)
mask_baby = meta_sample_df['sub_category_2'].str.contains('Baby', na=False)

# Update labels for baby-related subcategories
meta_sample_df.loc[mask_non_unique & mask_baby, 'sub_category_3'] = (
    meta_sample_df['sub_category_2'].str.split().str[:2].str.join(' ') + ' ' + meta_sample_df['sub_category_3']
)

# Update labels for other non-unique subcategories
meta_sample_df.loc[mask_non_unique & ~mask_baby, 'sub_category_3'] = (
    meta_sample_df['sub_category_2'].str.split().str[0] + ' ' + meta_sample_df['sub_category_3']
)

# Filter clothing-related data
clothing_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Clothing, Shoes & Accessories']

# Display updated category counts
print(clothing_df['sub_category_3'].value_counts(dropna=False))

sub_category_3
Key & Identification Accessories    1209
Men Tops & Tees                      859
Wallets & Bag Accessories            361
Women Tops & Tees                    237
Boys Tops & Tees                     172
                                    ... 
Baby Girls Costumes                    1
Boots                                  1
Baby Girls Skirts                      1
Loafers & Slip-Ons                     1
Leggings                               1
Name: count, Length: 86, dtype: int64


In [35]:
#Stationery & Party Supplies
stationary_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Stationery & Party Supplies']

print(stationary_df['sub_category_2'].value_counts(dropna=False))


sub_category_2
Stationery        2396
Party Supplies    1521
Pens & Pencils      16
Name: count, dtype: int64


In [36]:
#Moving Pens and Pencils under 'Stationery' as that seems like a suitable regrouping 

meta_sample_df.loc[meta_sample_df['sub_category_2'] == 'Pens & Pencils','sub_category_3'] = 'Pens & Pencils'

meta_sample_df.loc[meta_sample_df['sub_category_3'] == 'Pens & Pencils','sub_category_2'] = 'Stationery'

meta_sample_df.loc[meta_sample_df['sub_category_1'] == 'Stationery & Party Supplies','sub_category_2'].value_counts()

sub_category_2
Stationery        2412
Party Supplies    1521
Name: count, dtype: int64

In [37]:
#Beauty & Grooming
beauty_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Beauty & Grooming']
beauty_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Skin Care                 451
Hair Care                 424
Fragrance                 164
Makeup                     94
Tools & Accessories        91
Shaving & Hair Removal     50
Name: count, dtype: int64

In [38]:
# Sports & Outdoors
sports_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Sports & Outdoors']
sports_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Car & Vehicle Accessories    424
Hunting & Shooting           197
Sports & Fitness              65
Camping & Hiking              45
Fishing                       32
Cycling                        7
Name: count, dtype: int64

In [39]:
#Electronics Accessories
electronics_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Electronics Accessories']
electronics_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Laptop Accessories                235
Cell Phone Accessories            204
Fitness Tracker Bands & Straps     59
Computer Accessories               48
Camera & Photo Accessories         25
Name: count, dtype: int64

In [40]:
# Pet Supplies
pet_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Pet Supplies']
pet_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Cats    264
Dogs    187
Name: count, dtype: int64

In [41]:
#Toys & Games
toys_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Toys & Games']
toys_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Dolls, Toy Figures & Accessories    68
Learning & Education                57
Plushies & Stuffed Animals          53
Novelty & Gag Toys                  51
Puzzles                             47
Baby & Toddler Toys                 39
Pretend Play                        31
Lawn & Playground                   20
Puppets                              5
Musical Toy Instruments              4
Name: count, dtype: int64

In [42]:
#Baby
baby_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Baby']
baby_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Nursery                        249
Nursing & Feeding               26
Diaper Changing                 20
Health, Bathing & Skin Care     18
Pacifiers & Teethers            12
Carriers                         6
Name: count, dtype: int64

In [43]:
#Health & Personal Care
health_df = meta_sample_df[meta_sample_df['sub_category_1'] == 'Health & Personal Care']
health_df['sub_category_2'].value_counts(dropna=False)

sub_category_2
Personal Care            263
Wellness & Relaxation      3
Name: count, dtype: int64

In [44]:
#checking if sub_category_2 under differernt sub_catgory_1 have the same name
non_unique_subcat = []
for subcategory2 in meta_sample_df['sub_category_2'].unique():
    unique_cat = meta_sample_df.loc[meta_sample_df['sub_category_2'] == subcategory2,'sub_category_1'].nunique()
    if unique_cat != 1:
        print(f'{subcategory2} is a subcategory under {unique_cat} subcategories at level 1')
        non_unique_subcat.append(subcategory2)

print(f'{len(non_unique_subcat)} sub-categories under sub-category-1 have the same name.')

0 sub-categories under sub-category-1 have the same name.


In [45]:
#checking if sub_category_3 under differernt sub_catgory_2 have the same name
non_unique_subcat = []
for subcategory3 in meta_sample_df['sub_category_3'].dropna().unique():
    unique_cat = meta_sample_df.loc[meta_sample_df['sub_category_3'] == subcategory3,'sub_category_2'].nunique()
    if unique_cat != 1:
        print(f'{subcategory3} is a subcategory under {unique_cat} subcategories at level 2')
        non_unique_subcat.append(subcategory3)

print(f'{len(non_unique_subcat)} sub-categories under sub-category-2 have the same name.')

0 sub-categories under sub-category-2 have the same name.


In [46]:
#checking if sub_category_4 under differernt sub_catgory_3 have the same name
non_unique_subcat = []
for subcategory4 in meta_sample_df['sub_category_4'].dropna().unique():
    unique_cat = meta_sample_df.loc[meta_sample_df['sub_category_4'] == subcategory4,'sub_category_3'].nunique()
    if unique_cat != 1:
        print(f'{subcategory4} is a subcategory under {unique_cat} subcategories at level 3')
        non_unique_subcat.append(subcategory4)

print(f'{len(non_unique_subcat)} sub-categories under sub-category-3 have the same name.')

T-Shirts is a subcategory under 2 subcategories at level 3
Bandanas is a subcategory under 2 subcategories at level 3
Chains & Straps is a subcategory under 2 subcategories at level 3
Collars is a subcategory under 2 subcategories at level 3
Baseball Caps is a subcategory under 2 subcategories at level 3
ID Tags & Collar Accessories is a subcategory under 2 subcategories at level 3
Cases is a subcategory under 2 subcategories at level 3
Tank Tops is a subcategory under 2 subcategories at level 3
Display Stands is a subcategory under 2 subcategories at level 3
Essential Oils is a subcategory under 2 subcategories at level 3
Mobiles is a subcategory under 2 subcategories at level 3
Neckwear is a subcategory under 2 subcategories at level 3
12 sub-categories under sub-category-3 have the same name.


Some sub-category 4 labels are still duplicated across different sub-category 3 labels. Let’s rename these sub-category 4 labels to ensure that no two sub-category 4 labels from different sub-category 3 labels have the same name.

In [47]:
# Identify sub_category_4 values that are associated with multiple sub_category_3 values
subcat3_counts = meta_sample_df.groupby('sub_category_4')['sub_category_3'].nunique()

# Find non-unique subcategories (appearing under multiple sub_category_3)
non_unique_subcat = subcat3_counts[subcat3_counts > 1].index

# Create masks for updating sub_category_3 labels
mask_non_unique = meta_sample_df['sub_category_4'].isin(non_unique_subcat)
mask_special = meta_sample_df['sub_category_4'].str.contains(r'Mobiles|Essential Oils|Display Stands', na=False)

# Update labels for all but special subcategories
meta_sample_df.loc[mask_non_unique & ~mask_special, 'sub_category_4'] = (
    meta_sample_df['sub_category_2'].str.split().str[0] + ' ' + meta_sample_df['sub_category_4']
)

# Update labels individually for special subcategories

meta_sample_df.loc[(meta_sample_df['sub_category_4'].str.contains(r'Mobiles|Essential Oils')) & \
                   (meta_sample_df['sub_category_2'].str.contains('Nursery')),'sub_category_4'] = \
'Nursery ' + meta_sample_df['sub_category_4']

meta_sample_df.loc[(meta_sample_df['sub_category_4'] == 'Display Stands') & \
                   (meta_sample_df['sub_category_2'].str.contains('Home')),'sub_category_4'] = \
'Home ' + meta_sample_df['sub_category_4']

meta_sample_df.loc[meta_sample_df['sub_category_4'] == 'Essential Oils','sub_category_4'] = \
meta_sample_df['sub_category_3'].str.split().str[0] + ' ' + meta_sample_df['sub_category_4']


In [48]:
# sanity check that no two sub-category 4 labels from different sub-category 3 labels have the same name.
non_unique_subcat = []
for subcategory4 in meta_sample_df['sub_category_4'].dropna().unique():
    unique_cat = meta_sample_df.loc[meta_sample_df['sub_category_4'] == subcategory4,'sub_category_3'].nunique()
    if unique_cat != 1:
        print(f'{subcategory4} is a subcategory under {unique_cat} subcategories at level 3')
        non_unique_subcat.append(subcategory4)

print(f'{len(non_unique_subcat)} sub-categories under sub-category-3 have the same name.')


0 sub-categories under sub-category-3 have the same name.


In [49]:
non_unique_subcat = []
for subcategory5 in meta_sample_df['sub_category_5'].dropna().unique():
    unique_cat = meta_sample_df.loc[meta_sample_df['sub_category_5'] == subcategory5,'sub_category_4'].nunique()
    if unique_cat != 1:
        print(f'{subcategory5} is a subcategory under {unique_cat} subcategories at level 4')
        non_unique_subcat.append(subcategory4)

print(f'{len(non_unique_subcat)} sub-categories under sub-category-4 have the same name.')


Clothing Sets is a subcategory under 0 subcategories at level 4
Dresses is a subcategory under 0 subcategories at level 4
Bodysuits, Footies & Rompers is a subcategory under 0 subcategories at level 4
Oils is a subcategory under 2 subcategories at level 4
Tops & Tees is a subcategory under 0 subcategories at level 4
Pants is a subcategory under 0 subcategories at level 4
Costumes is a subcategory under 0 subcategories at level 4
Skirts is a subcategory under 0 subcategories at level 4
8 sub-categories under sub-category-4 have the same name.


 As you can see above, we still have sub_category_5 with similar names under two differnt sub_category_4/3. This shouldn't be a problem as we are almost never going to group products by sub_category5 because this column has lot of missing values.

 Now, we combine all sub-categories into a single column to represent the overall category of that product.

In [50]:
# This function combines non-null subcategory values from multiple hierarchical columns into a single
# 'all_subcategories' field. 
def combine_categories(row):
    """
    Combines non-null subcategory values into a single 'combined_category' field.

    Args:
    row (pd.Series): A row of the DataFrame.

    Returns:
    str: Concatenated string of non-null subcategories.
    """
    no_duplicate_combined = []
    combined = row['sub_category_1']  # Start with sub_category_1

    for ind in range(2, 6):  # Iterate through sub_category_2 to sub_category_5
        col_name = f'sub_category_{ind}'
        if pd.notna(row[col_name]):  # Check if the column value is not NaN
            combined += ' ' + row[col_name]  # Append the value
        # We can also add an else-break once we encounter a null subcategory, as the future subcategories should also be null.
    
     #Remove any duplicate words that exist in the combined string
    for x in re.split(r"[, ]+", combined): 
        if x not in no_duplicate_combined:
            no_duplicate_combined.append(x)
    return ' '.join(no_duplicate_combined)

In [51]:
#Create a new column - all_subcategories till the highest level of hierarchy available

# Apply the combine_categories function to the DataFrame
meta_sample_df['all_subcategories'] = meta_sample_df.apply(combine_categories,axis=1)

meta_sample_df['all_subcategories'].value_counts(dropna=False)

all_subcategories
Home & Kitchen Artwork Prints                            6823
Jewelry Necklaces Pendant                                3456
Jewelry Earrings Drop & Dangle                           2133
Home & Kitchen Décor Decorative Accessories Ornaments    1751
Home & Kitchen Décor Signs Plaques                       1421
                                                         ... 
Beauty & Grooming Tools Accessories                         1
Clothing Shoes & Accessories Women Hats Caps Visors         1
Clothing Shoes & Accessories Women Earmuffs                 1
Home & Kitchen Bedding Bed Pillows                          1
Clothing Shoes & Accessories Women Leggings                 1
Name: count, Length: 541, dtype: int64

We have a total of 500 unique categories in our dataset. For the purpose of modeling, we could consider using only those categories that have a sufficient number of items.

However, for now, we will keep all categories in the dataset and filter later as needed.

#### <a id='toc1_1_2_2_'></a>[Details Column](#toc0_)

In [52]:
meta_sample_df['details'].head()

0    {'Package Dimensions': '3.62 x 2.05 x 0.83 inc...
1    {'Department': 'unisex-adult', 'Date First Ava...
2    {'Package Dimensions': '11.2 x 8.8 x 0.2 inche...
3    {'Package Dimensions': '9 x 7 x 5 inches', 'Da...
4    {'Department': 'Womens', 'Date First Available...
Name: details, dtype: object

Details column contains dictionaries with keys describing different product features. Lets unpack this column into multiple columns.

In [53]:
#keeping records with non-empty values in details column
meta_sample_df = meta_sample_df[meta_sample_df['details'].map(bool)].reset_index(drop=True) #meta_sample_df[meta_sample_df['details'].map(len) > 0]

#Flatten nested dictionaries with 'details' column into separate columns.

details_df = pd.json_normalize(meta_sample_df['details']) #details_df = pd.DataFrame.from_dict(meta_sample_df['details'].to_list())

#joining the original df
meta_sample_expanded_df = pd.concat([meta_sample_df, details_df], axis=1)

meta_sample_expanded_df.columns


Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'sub_category_1', 'sub_category_2',
       'sub_category_3', 'sub_category_4', 'sub_category_5', 'sub_category_6',
       'all_subcategories', 'Package Dimensions', 'Department',
       'Date First Available', 'Manufacturer recommended age', 'Item Weight',
       'Manufacturer', 'Product Dimensions', 'Item model number',
       'Country of Origin', 'Is Discontinued By Manufacturer', 'Color',
       'Brand', 'Theme', 'Mounting Type', 'Shape',
       'Best Sellers Rank.Handmade Products',
       'Best Sellers Rank.Handmade Picture Frames',
       'Best Sellers Rank.Photo Albums, Frames & Accessories', 'Material',
       'Batteries', 'Batteries required',
       'Best Sellers Rank.Handmade Signs & Plaques',
       'Best Sellers Rank.Handmade Prints',
       'Best Sellers Rank.Posters & Prints', 'Material 

In [54]:
#Checking duplicated columns
print(f"There are {sum(meta_sample_expanded_df.columns.duplicated())} duplicated columns in the dataframe.")

There are 0 duplicated columns in the dataframe.


Columns Product Dimensions, Item Weight, contain information about the item wherease Package Dimensions contains information about the Package. Since there are a lot of null values in Product Dimensions/ Item Weight , we will be dropping them.

In [55]:
#Checking null values
meta_sample_expanded_df.isna().mean()*100

main_category                                             0.000000
title                                                     0.000000
average_rating                                            0.000000
rating_number                                             0.000000
features                                                  0.000000
description                                               0.000000
price                                                     0.000000
images                                                    0.000000
videos                                                    0.000000
store                                                     0.000000
categories                                                0.000000
details                                                   0.000000
parent_asin                                               0.000000
sub_category_1                                            0.000000
sub_category_2                                            0.00

Most of the newly created columns appear to have more than 99% data missing and will likely not contribute meaningful information to the model.

In [56]:
#Delete columns that have more than 20% of rows missing 
meta_sample_expanded_df = meta_sample_expanded_df.dropna(thresh= 0.20*len(meta_sample_expanded_df),axis=1)

meta_sample_expanded_df.columns.to_list()

['main_category',
 'title',
 'average_rating',
 'rating_number',
 'features',
 'description',
 'price',
 'images',
 'videos',
 'store',
 'categories',
 'details',
 'parent_asin',
 'sub_category_1',
 'sub_category_2',
 'sub_category_3',
 'sub_category_4',
 'all_subcategories',
 'Package Dimensions',
 'Department',
 'Date First Available']

In [57]:
#checking date first available column
meta_sample_expanded_df['Date First Available'].head(10)

0     October 27, 2015
1     January 10, 2018
2     November 5, 2019
3     January 26, 2016
4    November 14, 2016
5       March 30, 2019
6        April 5, 2019
7      August 23, 2018
8      August 22, 2016
9      October 1, 2016
Name: Date First Available, dtype: object

In [58]:
#convert ` Date First Available` column to datetime, coercing errors to NaT
meta_sample_expanded_df['date_first_available'] = pd.to_datetime(meta_sample_expanded_df['Date First Available'],errors='coerce')

# Drop rows with NaT values in the column
meta_sample_expanded_df = meta_sample_expanded_df.dropna(subset=['date_first_available']).reset_index(drop=True)

#Delete redundant column 
meta_sample_expanded_df.drop(columns = 'Date First Available',inplace=True)

#checking column data types
meta_sample_expanded_df.dtypes


main_category                   object
title                           object
average_rating                 float64
rating_number                    int64
features                        object
description                     object
price                          float64
images                          object
videos                          object
store                           object
categories                      object
details                         object
parent_asin                     object
sub_category_1                  object
sub_category_2                  object
sub_category_3                  object
sub_category_4                  object
all_subcategories               object
Package Dimensions              object
Department                      object
date_first_available    datetime64[ns]
dtype: object

In [59]:
#Inspecting Package Dimensions column
meta_sample_expanded_df['Package Dimensions'].head(10)

0    3.62 x 2.05 x 0.83 inches; 0.46 Ounces
1                                       NaN
2      11.2 x 8.8 x 0.2 inches; 3.21 Ounces
3                          9 x 7 x 5 inches
4                                       NaN
5                         10 x 8 x 1 inches
6    3.78 x 2.87 x 0.75 inches; 0.18 Ounces
7                                       NaN
8                                       NaN
9       4.4 x 4.4 x 4.3 inches; 6.38 Ounces
Name: Package Dimensions, dtype: object

In [60]:
#Splitting Package Dimensions into dimensions and weight columns
meta_sample_expanded_df[['package_dimensions_inches','package_weight_ounces']] = meta_sample_expanded_df['Package Dimensions'].str.lower().str.split('; ',expand=True).replace({None: np.nan})

# Cleaning the columns further for non missing rows
rows_not_null_dim, rows_not_null_wt = meta_sample_expanded_df['package_dimensions_inches'].notna(), meta_sample_expanded_df['package_weight_ounces'].notna()

meta_sample_expanded_df.loc[rows_not_null_wt,'package_weight_ounces'] = (
    meta_sample_expanded_df.loc[rows_not_null_wt,'package_weight_ounces']
    .apply(lambda x: float(x.replace('ounces', '').strip()) if 'ounces' in x else float(x.replace('pounds', '').strip()) * 16)  # Handle Pounds or Ounces
)
meta_sample_expanded_df['package_weight_ounces'] = meta_sample_expanded_df['package_weight_ounces'].astype('float')


meta_sample_expanded_df[['package_length_inches', 'package_width_inches', 'package_height_inches']] = (
                                                                        meta_sample_expanded_df['package_dimensions_inches']
                                                                        .str.replace('inches', '')  
                                                                        .str.strip()  
                                                                        .str.split(' x ',expand=True)
                                                                        .astype('float')
                                                                    )
#Delete redundant column 
meta_sample_expanded_df.drop(columns = 'package_dimensions_inches', inplace=True)

In [61]:
#checking null values
meta_sample_expanded_df.isna().mean().loc[lambda x: x>0]

sub_category_3           0.050926
sub_category_4           0.745911
Package Dimensions       0.598765
Department               0.570454
package_weight_ounces    0.613105
package_length_inches    0.598765
package_width_inches     0.598765
package_height_inches    0.598765
dtype: float64

#### <a id='toc1_1_2_3_'></a>[Department Column](#toc0_)

In [62]:
print(f'Unique Department Values in the dataset:')
print(meta_sample_expanded_df['Department'].str.lower().value_counts().index.unique().to_list())

Unique Department Values in the dataset:
['womens', 'unisex-adult', 'women', 'mens', 'girls', 'unisex adult', 'unisex-child', 'unisex', 'unisex-baby', 'women, girls', 'men', 'boys', 'baby-girls', 'women, men', 'unisex adult, unisex child', 'women-girls', 'baby-boys', 'female', 'woman', 'girls, women', 'men, women', 'women/girls', "women,women's,ladies,girl,girl's", 'women girls', 'women, men, girls, boys', 'women, men, unisex', 'women,girls', 'women,men', 'women,men,unisex', 'women or men', 'women, girl', 'women and girls', 'baby', 'women, man', 'women, girls, and babies', 'women men', 'unisex adult, unisex children', 'women, girls, unisex', 'both', 'girls, women, men, unisex adult', 'women, girls,', 'any body', 'boys,girls,unisex adult', 'baby girl', 'girls, woman', 'women and girls.', 'women, men, boys, girls', 'women/girls/unisex', 'cheerleader', 'unisex adult & child', 'teen', 'everyone', 'women, girls, men, unisex adult', 'men and women', 'dance', 'unisex adult,women', 'girls,wome

In [63]:
# Function to clean different categories in the Department column 

def clean_and_categorize(text):
    """
    Cleans the text in department column to create cleaner categories.

    Args:
        text (str): The input text from Department column.

    Returns:
        str: The cleaned department
    """

    # Step 1: Clean text
    clean_text = text.lower()
    clean_text = re.sub(r'[^a-z /s]', ' ', clean_text).strip()  # Remove non-alphabetic characters
    clean_text = re.sub(r'\ss', '', clean_text)  # Remove space before 's
    clean_text = re.sub(r'wo\w+', 'women', clean_text)  # Normalize 'women'
    clean_text = re.sub(r'man', 'men', clean_text)  # Normalize 'men'
    clean_text = re.sub(r'\/', ' ', clean_text)  # Replace '/' with space
    clean_text = re.sub(r'\s+', ',', clean_text)  # Replace spaces with commas
    
    # Step 2: Clean categories
    category_map = {
        r'\b(girls?|ladies|female?|womens?)\b': 'women',
        r'\b(boys?|male?|mens?)\b': 'men',
        r'\b(both|unisex|uniisex|any\w*|everyone|all)\b': 'unisex',
        r'\b(dance|volleyball|field|hockey|soccer|softball|swim|cheerlead\w+|sports)\b': 'sports',
        r'\b(bab\w+|infant|toddler|kids?|teens?|child\w*)\b': 'kids',
        r'\b(pet|brown|re|communion|wedding)\b': 'misc'
    }

    for pattern, replacement in category_map.items():
        clean_text = re.sub(pattern, replacement, clean_text)

    # Step 3: Categorize based on cleaned values
    set_cat = set(clean_text.split(','))
    
    special_map = {
    frozenset({'adult', 'kids'}): 'unisex',
    frozenset({'kids', 'men', 'women'}): 'unisex',
    frozenset({'kids', 'unisex'}): 'unisex',
    frozenset({'women', 'men'}): 'unisex',
                   }

    priority_list = ['kids', 'unisex', 'women', 'men', 'sports', 'misc']

    #check special_map first
    for key_set,category in special_map.items():
        if key_set <= set_cat:
            return(category)

    # Check priority categories in order
    for category in priority_list:
        if category in set_cat:
            return category


In [64]:
# Apply the clean_and_categorize function to the 'Department' column

meta_sample_expanded_df['department'] = (meta_sample_expanded_df['Department']
                                                .apply(lambda x: clean_and_categorize(x) if pd.notnull(x) else x)
                                                )
#checking distinct values in the column
meta_sample_expanded_df['department'].value_counts(dropna=False)

department
NaN       26335
women     11707
unisex     6707
men        1269
kids        123
sports       18
misc          6
Name: count, dtype: int64

In [65]:
#Rename columns
meta_sample_expanded_df = meta_sample_expanded_df.rename(
                          columns={'Department': 'orig_department',
                                   'videos':'videos_product',
                                   'description':'description_product'})

In [66]:
# Pickle the Cleaned DataFrame
meta_sample_expanded_df.to_pickle('../data/meta_sample_handmade.pkl')

### <a id='toc1_1_3_'></a>[Loading Review dataset](#toc0_)

Reviews dataset contains full review text data including user_id of the reviewer and the rating they provided. Make sure to download the json file from [here](https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw) before loading it into the dataframe.

In [67]:
# Load the Reviews json file into a dataframe
review_df = pd.read_json('../data/Handmade_Products.jsonl',lines=True)

# Display the first few rows to get a preview of the 'reviews_df' DataFrame
review_df.head() 

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Beautiful colors,I bought one for myself and one for my grandda...,[],B08GPJ1MSN,B08GPJ1MSN,AF7OANMNHQJC3PD4HRPX2FATECPA,2021-05-21 14:31:35.111,1,True
1,5,You simply must order order more than one!,I’ve ordered three bows so far. Have not been ...,[],B084TWHS7W,B084TWHS7W,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,2020-04-24 21:15:46.965,0,True
2,5,Great,As pictured. Used a frame from the dollar stor...,[],B07V3NRQC4,B07V3NRQC4,AEYORY2AVPMCPDV57CE337YU5LXA,2020-06-06 13:09:11.297,0,True
3,5,Well made and so beautiful,"This is beyond beautiful. So shiny, the size ...",[],B071ZMDK26,B071ZMDK26,AEINY4XOINMMJCK5GZ3M6MMHBN6A,2019-06-02 01:14:39.784,2,True
4,5,Smells just like the real thing!,Oh wow what a pleasant surprise! This smells g...,[],B01MPVZ4YP,B01MPVZ4YP,AGCPAPUHXYA3EEIL2KGSQTGO5HRA,2019-01-08 00:12:11.674,1,True


In [68]:
# Inspecting review_df and check its concise summary
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664162 entries, 0 to 664161
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   rating             664162 non-null  int64         
 1   title              664162 non-null  object        
 2   text               664162 non-null  object        
 3   images             664162 non-null  object        
 4   asin               664162 non-null  object        
 5   parent_asin        664162 non-null  object        
 6   user_id            664162 non-null  object        
 7   timestamp          664162 non-null  datetime64[ns]
 8   helpful_vote       664162 non-null  int64         
 9   verified_purchase  664162 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 46.2+ MB


In [69]:
# Print the shape of the 'review_df' DataFrame
print(f"The shape of the review dataset is {review_df.shape[0]} by {review_df.shape[1]}.")

The shape of the review dataset is 664162 by 10.


In [70]:
#check number of null values explicitly
review_df.isnull().sum()

rating               0
title                0
text                 0
images               0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
dtype: int64

The reviews dataset contains no missing values, so no major preprocessing steps are required. However, we further analyze this dataset when examining early reviews of the products.

In [71]:
#rename columns
review_df.rename(columns={'rating':'rating_by_user','text':'text_review',
                          'timestamp':'time_of_review','asin':'product_id',
                         },inplace=True)

In [72]:
# Pickle the Final DataFrame
review_df.to_pickle('../data/review_sample_handmade.pkl')

### <a id='toc1_1_4_'></a>[Conclusion](#toc0_)

In this notebook, we loaded and cleaned the metadata to extract meaningful product attributes. Next, we will preprocess various metadata columns to ensure they are suitable for modeling.